In [1]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender imageio==2.4.1 atari-py==0.2.6 gym==0.17.3
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install ptan pytorch-ignite pybullet
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.2/790.2 KB 46.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.9 MB/s eta 0:00:00
Reason for being yanked: re-release with new wheels
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=17ed6c9acae329a90eebed9dceb481c00a036dca50cf4b6b9f5cd183f176232a
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
  Created wheel for atari-py: filename=atari_py-0.2.6-cp38-cp38-linux_x86_64.whl size=3092751 sha256=87edb

In [2]:
import collections
from typing import Union
import math
import random
from copy import deepcopy
from typing import Optional

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torch.distributions as distr
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
import pybullet_envs
from colabgymrender.recorder import Recorder
from fastprogress import progress_bar as pb
import matplotlib.pyplot as plt

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2383872/45929032 bytes (5.2%)4964352/45929032 bytes (10.8%)7667712/45929032 bytes (16.7%)11018240/45929032 bytes (24.0%)13975552/45929032 bytes (30.4%)16801792/45929032 bytes (36.6%)18857984/45929032 bytes (41.1%)20709376/45929032 bytes (45.1%)23298048/45929032 bytes (50.7%)25010176/45929032 bytes (54.5%)27574272/45929032 bytes (60.0%)30097408/45929032 bytes (65.5%)

In [3]:
class NoisyLinear(nn.Linear):

    def __init__(self, in_features, out_features, sigma_init=0.017, bias=True):
        super(NoisyLinear, self).__init__(in_features, out_features, bias=bias)
        w = torch.full((out_features, in_features), sigma_init)
        self._sigma_weight = nn.Parameter(w)
        z = torch.zeros(out_features, in_features)
        self.register_buffer("epsilon_weight", z)
        if bias:
            w = torch.full((out_features,), sigma_init)
            self._sigma_bias = nn.Parameter(w)
            z = torch.zeros(out_features)
            self.register_buffer("epsilon_bias", z)
        self.reset_parameters()

    def reset_parameters(self):
        std = math.sqrt(3 / self.in_features)
        self.weight.data.uniform_(-std, std)
        self.bias.data.uniform_(-std, std)

    def forward(self, input):
        self.epsilon_weight.normal_()
        bias = self.bias
        if bias is not None:
            self.epsilon_bias.normal_()
            bias = bias + self._sigma_bias * \
                   self.epsilon_bias.data
        v = self._sigma_weight * self.epsilon_weight.data + \
            self.weight
        return F.linear(input, v, bias)

In [4]:
class SACActor(nn.Module):

    def __init__(self, obs_size, act_size, hidden_size: int = 64):
        super(SACActor, self).__init__()

        self.mu = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, act_size),
            nn.Tanh(),
        )
        self.logstd = nn.Parameter(torch.zeros(act_size))

    def forward(self, x):
        return self.mu(x)

In [5]:
class SACCritic(nn.Module):

    def __init__(self, obs_size, hidden_size: int = 64):
        super(SACCritic, self).__init__()

        self.value = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, x):
        return self.value(x)

In [6]:
class SACTwinQ(nn.Module):

    def __init__(self, obs_size, act_size, hidden_size: int = 64):
        super(SACTwinQ, self).__init__()

        self.q1 = nn.Sequential(
            nn.Linear(obs_size + act_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

        self.q2 = nn.Sequential(
            nn.Linear(obs_size + act_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

    def forward(self, obs, act):
        x = torch.cat([obs, act], dim=1)
        return self.q1(x), self.q2(x)

In [7]:
class NStepPriorityReplayBuffer:

    def __init__(
        self,
        max_size: int,
        prob_alpha: float = 0.6,
        beta_start: float = 0.4,
        beta_frames: float = 100000,
        n_step: int = 4,
        gamma: float = 0.99,
    ):
        self._prob_alpha = prob_alpha
        self._max_size = max_size
        self._pos = 0
        self._buf = []
        self._priorities = np.zeros((max_size,), dtype=np.float32)
        self._beta_start = beta_start
        self._beta = beta_start
        self._beta_frames = beta_frames
        self._n_step = n_step
        self._gamma = gamma
        self._total_discounted_rewards = np.array([np.nan]*max_size)
        self._last_states = [np.nan]*max_size

    def update_bata(self, idx) -> None:
        beta = self._beta_start + idx * (1.0 - self._beta_start) / self._beta_frames
        self._beta = min(1.0, beta)
        return self._beta

    def __len__(self):
        return len(self._buf)

    def append(
        self,
        state: np.ndarray,
        action: int,
        reward: Union[int, float],
        done: bool,
        next_state: np.ndarray,
    ) -> None:
        max_prio = self._priorities.max() if self._buf else 1.0
        if len(self._buf) < self._max_size:
            self._buf.append(
                (state, action, reward, done, next_state)
            )
        else:
            self._buf[self._pos] = (state, action, reward, done, next_state)
        self._priorities[self._pos] = max_prio

        if len(self._buf) >= self._n_step:
            dis_r = 0.0
            last_state = self._buf[self._pos][0]
            for i in range(self._n_step):
                state, _, r, done, _ = self._buf[self._pos - i]
                dis_r = r + self._gamma * dis_r
                if done:
                    last_state = state
                    dis_r = r
                self._total_discounted_rewards[self._pos - i] = dis_r
                self._last_states[self._pos - i] = last_state
            
            for i in range(self._n_step-1):
                done = self._buf[self._pos - i][3]
                if done:
                    break
                self._total_discounted_rewards[self._pos - i] = np.nan
                self._last_states[self._pos - i] = np.nan

        self._pos = (self._pos + 1) % self._max_size

    def sample(self, size: int):
        sample_target_indices = np.where(~np.isnan(self._total_discounted_rewards[:len(self._buf)]))[0]
        # prios = self._priorities[sample_target_indices]  #self._priorities if len(self._buf) == self._max_size else self._priorities[:self._pos]
        prios = self._priorities
        probs = prios * self._prob_alpha
        # probs /= np.nan_to_num(probs, 0.0).sum()
        probs /= probs[sample_target_indices].sum()
        sampled_indices = np.random.choice(
            sample_target_indices,
            # np.where(~np.isnan(self._total_discounted_rewards[:len(self._buf)]))[0],
            size, p=probs[sample_target_indices]
        )
        states, actions, rewards, dones, next_states = zip(*[self._buf[idx] for idx in sampled_indices])
        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        dones = np.array(dones)
        next_states = np.array(next_states)
        total_discounted_rewards = self._total_discounted_rewards[sampled_indices]
        last_states = np.stack([self._last_states[idx] for idx in sampled_indices])
        total = len(self._buf)
        weights = np.array((total * probs[sampled_indices]) ** (-self._beta), dtype=np.float32)
        # weights = np.array((total * probs) ** (-self._beta), dtype=np.float32)
        weights /= weights.max()
        return states, actions, rewards, dones, total_discounted_rewards, last_states, sampled_indices, weights

    def update_priorities(self, sample_indices: np.ndarray, sample_priorities: np.ndarray) -> None:
        self._priorities[sample_indices] = sample_priorities

    @property
    def gamma(self) -> float:
        return self._gamma

    @property
    def n_step(self) -> float:
        return self._n_step

In [8]:
class Agent:

    def __init__(
        self,
        env,
        exp_buffer: NStepPriorityReplayBuffer,
        act_net: nn.Module,
        crt_net: nn.Module,
        twinq_net:  nn.Module,
        epsilon_start: float = 1.0,
        epsilon_final: float = 0.01,
        epsilon_decay_last_step: int = 200000,
        tgt_sync_steps: int = 10000,
        learning_rate_acts: float = 1e-4,
        learning_rate_vals: float = 1e-4,
        adam_eps: float = None,
        sac_entropy_alpha: float = 0.1,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self._env = env
        self._exp_buffer = exp_buffer
        self._act_net = act_net
        self._crt_net = crt_net
        self._twinq_net = twinq_net
        self._tgt_act_net = deepcopy(act_net)
        self._tgt_crt_net = deepcopy(crt_net)
        for p in self._tgt_act_net.parameters():
            p.requires_grad = False
        for p in self._tgt_crt_net.parameters():
            p.requires_grad = False
        self._epsilon_start = epsilon_start
        self._epsilon_final = epsilon_final
        self._epsilon_decay_last_step = epsilon_decay_last_step
        self._epsilon = epsilon_start
        self._device = device
        self._total_step = 0
        self._total_trained_samples = 0
        self._tgt_sync_steps = tgt_sync_steps
        adam_kwargs = {}
        if adam_eps is not None:
            adam_kwargs['eps'] = adam_eps
        self._crt_optimizer = torch.optim.Adam(self._crt_net.parameters(), lr=learning_rate_vals, **adam_kwargs)
        self._act_optimizer = torch.optim.Adam(self._act_net.parameters(), lr=learning_rate_acts, **adam_kwargs)
        self._twinq_optimizer = torch.optim.Adam(self._twinq_net.parameters(), lr=learning_rate_vals, **adam_kwargs)
        self._sac_entropy_alpha = sac_entropy_alpha
        self._reset_episode()

    def _reset_episode(self):
        self._state = self._env.reset()
        self._total_reward = 0.0

    @torch.no_grad()
    def play_step(self, epsilon: Optional[float] = None, sync_target: bool = True):
        if epsilon is None:
            epsilon = self._epsilon
        done_reward = None

        if np.random.random() < epsilon:
            action = self._env.action_space.sample()
        else:
            state_a = np.array([self._state], copy=False)
            state_v = torch.tensor(state_a).float().to(self._device)
            mu_v = self._act_net(state_v)
            action = mu_v.squeeze(dim=0).data.cpu().numpy()
            action = np.clip(action, -1, 1)

        next_state, reward, is_done, _ = self._env.step(action)
        self._total_reward += reward

        self._exp_buffer.append(
            self._state, action, reward, is_done, next_state
        )
        self._state = next_state
        if is_done:
            done_reward = self._total_reward
            self._reset_episode()

        self._total_step += 1
        self._update_epsilon(self._total_step)
        self._exp_buffer.update_bata(self._total_step)

        # if self._total_step % self._tgt_sync_steps == 0 and sync_target:
        #     self._alpha_sync(self._act_net, self._tgt_act_net, alpha=1 - 1e-3)
        #     self._alpha_sync(self._crt_net, self._tgt_crt_net, alpha=1 - 1e-3)
        #     print(f'synced target net')

        return done_reward

    def train(self, n_iter: int = 1, batch_size: int = 32) -> None:
        n_step_gamma = self._exp_buffer.gamma ** self._exp_buffer.n_step
        for i in range(n_iter):
            states, actions, rewards, dones, total_discounted_rewards, \
                last_states, sampled_indices, weights = self._exp_buffer.sample(batch_size)
            states_v = torch.tensor(states).float().to(self._device)
            actions_v = torch.tensor(actions).to(self._device)
            discounted_rewards_v = torch.tensor(total_discounted_rewards).to(self._device)
            done_mask = torch.BoolTensor(dones).to(self._device)
            weights_v = torch.tensor(weights).to(self._device)
            last_states_v = torch.tensor(last_states).float().to(self._device)

            # last_act_v = self._tgt_act_net(last_states_v)
            q_last_v = self._tgt_crt_net(last_states_v)
            q_last_v[done_mask] = 0.0
            q_ref_v = discounted_rewards_v.unsqueeze(dim=-1) + \
                        q_last_v * n_step_gamma

            mu_v = self._act_net(states_v)
            act_dist = distr.Normal(mu_v, torch.exp(self._act_net.logstd))
            acts_v = act_dist.sample()
            q1_v, q2_v = self._twinq_net(states_v, acts_v)
            # element-wise minimum
            vals_ref_v = torch.min(q1_v, q2_v).squeeze() - \
                        self._sac_entropy_alpha * act_dist.log_prob(acts_v).sum(dim=1)

            # train TwinQ
            self._twinq_optimizer.zero_grad()
            q1_v, q2_v = self._twinq_net(states_v, actions_v)
            q1_loss_v = F.mse_loss(q1_v.squeeze().float(),
                                    q_ref_v.squeeze().detach().float())
            q2_loss_v = F.mse_loss(q2_v.squeeze().float(),
                                    q_ref_v.squeeze().detach().float())
            q_loss_v = q1_loss_v + q2_loss_v
            q_loss_v.backward()
            self._twinq_optimizer.step()

            # Critic
            self._crt_optimizer.zero_grad()
            val_v = self._crt_net(states_v)
            v_loss_v = F.mse_loss(val_v.squeeze().float(),
                                    vals_ref_v.squeeze().detach().float())
            v_loss_v.backward()
            self._crt_optimizer.step()

            # Actor
            self._act_optimizer.zero_grad()
            acts_v = self._act_net(states_v)
            q_out_v, _ = self._twinq_net(states_v, acts_v)
            act_loss = -q_out_v.mean()
            act_loss.backward()
            self._act_optimizer.step()

            # self._alpha_sync(self._act_net, self._tgt_act_net, alpha=1 - 1e-3)
            self._alpha_sync(self._crt_net, self._tgt_crt_net, alpha=1 - 1e-3)

            # self._exp_buffer.update_priorities(sampled_indices, prios)
            self._total_trained_samples += batch_size

    def initial_exploration(self, n_steps: int = 10000, epsilon: float = 1.0) -> None:
        eps_bak = self._epsilon
        try:
            for i in pb(range(n_steps)):
                self._epsilon = epsilon
                self.play_step(sync_target=False)
        finally:
            self._total_step = 0
            self._epsilon = eps_bak

    def _update_epsilon(self, step_index: int) -> None:
        self._epsilon = max(
            self._epsilon_final,
            self._epsilon_start - step_index / self._epsilon_decay_last_step
        )

    def _sync(self, net, tgt_net):
        tgt_net.load_state_dict(net.state_dict())

    def _alpha_sync(self, net, tgt_net, alpha):
        assert isinstance(alpha, float)
        assert 0.0 < alpha <= 1.0
        state = net.state_dict()
        tgt_state = tgt_net.state_dict()
        for k, v in state.items():
            tgt_state[k] = tgt_state[k] * alpha + (1 - alpha) * v
        tgt_net.load_state_dict(tgt_state)


In [9]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        print(observation.shape)
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose(
            [T.Resize(self.shape), T.Normalize(0, 255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation


class LazyFramesToNumpy(gym.ObservationWrapper):
    def __init__(self, env):
        super(LazyFramesToNumpy, self).__init__(env)

    def observation(self, observation):
        return observation.__array__()

In [10]:
SEED = 77

In [11]:
random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!rm -rf /content/video

env = gym.make("MinitaurBulletEnv-v0")
# env = SkipFrame(env, skip=4)
# env = GrayScaleObservation(env)
# env = ResizeObservation(env, shape=224)
# env = FrameStack(env, num_stack=4)
# env = LazyFramesToNumpy(env)
directory = './video'
# env = Recorder(env, directory)

current_dir=/usr/local/lib/python3.8/dist-packages/pybullet_envs/bullet
urdf_root=/usr/local/lib/python3.8/dist-packages/pybullet_data


  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



In [12]:
env.observation_space.shape, env.action_space

((28,), Box(-1.0, 1.0, (8,), float32))

In [13]:
env.action_space.sample()

array([ 0.25194114, -0.49410233, -0.57637924, -0.06971877, -0.8521081 ,
       -0.9243232 ,  0.12026657,  0.24457712], dtype=float32)

In [14]:
state = env.reset()

In [15]:
state

array([ 1.49275966e+00,  1.48147845e+00,  1.48712334e+00,  1.47808718e+00,
        1.49588935e+00,  1.50007679e+00,  1.49504475e+00,  1.49626286e+00,
        8.84849083e-01,  8.97834931e-01,  9.05268987e-01,  9.14548288e-01,
        8.22842201e-01,  8.02721571e-01,  8.39267658e-01,  8.22955943e-01,
        4.92169263e-01,  5.81561550e-01,  5.34936073e-01,  6.06559382e-01,
        4.74579163e-01,  4.43823720e-01,  4.79211186e-01,  4.71839865e-01,
       -1.64014475e-03,  9.45274037e-04, -2.28336747e-04,  9.99998182e-01])

In [34]:
act_net = SACActor(
    env.observation_space.shape[0],
    env.action_space.shape[0]
).to(device)
crt_net = SACCritic(
    env.observation_space.shape[0],
    env.action_space.shape[0]
).to(device)
twinq_net = SACTwinQ(
    env.observation_space.shape[0],
    env.action_space.shape[0]
).to(device)

In [35]:
act_net

SACActor(
  (mu): Sequential(
    (0): Linear(in_features=28, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=8, bias=True)
    (5): Tanh()
  )
)

In [36]:
exp_buffer = NStepPriorityReplayBuffer(
    max_size=30000,
    prob_alpha=0.6,
    beta_start=0.4,
    beta_frames=30000, #100000,
    n_step=20,
    gamma=0.99,
)

In [37]:
agent = Agent(
    env=env,
    exp_buffer=exp_buffer,
    crt_net=crt_net,
    act_net=act_net,
    twinq_net=twinq_net,
    epsilon_start=0.2,
    epsilon_final=0.002,
    epsilon_decay_last_step=50000, #200000,
    # tgt_sync_steps=1000,
    learning_rate_acts=1e-4,
    learning_rate_vals=1e-4,
    device=device
)

In [38]:
act_net(torch.tensor(state).unsqueeze(0).float().to(device))

tensor([[ 0.3702, -0.1791, -0.1584, -0.0662,  0.1663, -0.0960,  0.0526, -0.2133]],
       grad_fn=<TanhBackward0>)

In [39]:
crt_net(
    torch.tensor(state).unsqueeze(0).float().to(device),
)

tensor([[-0.3097]], grad_fn=<AddmmBackward0>)

In [40]:
agent.initial_exploration(n_steps=10000)

In [41]:
!rm -rf /content/video/*

In [ ]:
# exp_buffer = NStepPriorityReplayBuffer(
#     max_size=30000,
#     prob_alpha=0.6,
#     beta_start=0.4,
#     beta_frames=30000, #100000,
#     n_step=20,
#     gamma=0.99,
# )
# agent = Agent(
#     env=env,
#     exp_buffer=exp_buffer,
#     crt_net=crt_net,
#     act_net=act_net,
#     twinq_net=twinq_net,
#     epsilon_start=0.2,
#     epsilon_final=0.002,
#     epsilon_decay_last_step=50000, #200000,
#     # tgt_sync_steps=1000,
#     learning_rate_acts=1e-4,
#     learning_rate_vals=1e-4,
#     device=device
# )

episode = 0

while True:

    for stp in range(1):
        done_reward = agent.play_step()
        if done_reward is not None:
            print(f'episode : {episode}, done reward : {done_reward}, total_step : {agent._total_step}, cur_epsilon : {agent._epsilon}')
            episode += 1

    agent.train(n_iter=1, batch_size=64)

ストリーミング出力は最後の 5000 行に切り捨てられました。
episode : 10640, done reward : -0.07125306207967567, total_step : 1681051, cur_epsilon : 0.002
episode : 10641, done reward : -0.18524448019250175, total_step : 1682051, cur_epsilon : 0.002
episode : 10642, done reward : -0.540628515016501, total_step : 1682433, cur_epsilon : 0.002
episode : 10643, done reward : 0.0880253700604977, total_step : 1682480, cur_epsilon : 0.002
episode : 10644, done reward : -0.33525978766387626, total_step : 1683178, cur_epsilon : 0.002
episode : 10645, done reward : -0.19037527913741312, total_step : 1683591, cur_epsilon : 0.002
episode : 10646, done reward : -0.020081576372571793, total_step : 1683625, cur_epsilon : 0.002
episode : 10647, done reward : 0.027800512347027313, total_step : 1683757, cur_epsilon : 0.002
episode : 10648, done reward : -1.2075288703913145, total_step : 1684757, cur_epsilon : 0.002
episode : 10649, done reward : -0.8877684835062013, total_step : 1685179, cur_epsilon : 0.002
episode : 10650, done r

In [33]:
# exp_buffer = NStepPriorityReplayBuffer(
#     max_size=30000,
#     prob_alpha=0.6,
#     beta_start=0.4,
#     beta_frames=30000, #100000,
#     n_step=15,
#     gamma=0.99,
# )
# agent = Agent(
#     env=env,
#     exp_buffer=exp_buffer,
#     crt_net=crt_net,
#     act_net=act_net,
#     twinq_net=twinq_net,
#     epsilon_start=0.1,
#     epsilon_final=0.002,
#     epsilon_decay_last_step=50000, #200000,
#     # tgt_sync_steps=1000,
#     learning_rate_acts=1e-4,
#     learning_rate_vals=1e-4,
#     device=device
# )

episode = 0

while True:

    for stp in range(1):
        done_reward = agent.play_step()
        if done_reward is not None:
            print(f'episode : {episode}, done reward : {done_reward}, total_step : {agent._total_step}, cur_epsilon : {agent._epsilon}')
            episode += 1

    agent.train(n_iter=1, batch_size=64)

episode : 0, done reward : -0.9184538190121604, total_step : 194, cur_epsilon : 0.09612000000000001
episode : 1, done reward : 0.028865178903864648, total_step : 251, cur_epsilon : 0.09498000000000001
episode : 2, done reward : -0.0068176277573966946, total_step : 279, cur_epsilon : 0.09442
episode : 3, done reward : -0.0009995136840012394, total_step : 313, cur_epsilon : 0.09374
episode : 4, done reward : 0.011286421629160687, total_step : 348, cur_epsilon : 0.09304000000000001
episode : 5, done reward : -0.016988154303184063, total_step : 377, cur_epsilon : 0.09246
episode : 6, done reward : -0.0012692403574853014, total_step : 422, cur_epsilon : 0.09156
episode : 7, done reward : 0.05328991002824718, total_step : 462, cur_epsilon : 0.09076000000000001
episode : 8, done reward : 0.005696926650304132, total_step : 494, cur_epsilon : 0.09012
episode : 9, done reward : -0.019222902226705578, total_step : 523, cur_epsilon : 0.08954000000000001
episode : 10, done reward : -0.0321748747828

KeyboardInterrupt: ignored